## Introduction

This notebook demonstrates the final phase of a data engineering pipeline for storing cleaned and processed Google Play review data in an Oracle XE database. The pipeline connects to an Oracle database, inserts cleaned review data from CSV files, validates the inserted data by querying row counts, and generates an SQL dump for version control on GitHub. This workflow simulates real-world enterprise data engineering practices, particularly in banking environments where Oracle databases are prevalent.

---

### Setup Environment

The following code sets up the Python environment by adding parent directory to path for script imports and importing required libraries along with custom scripts that handle data insertion and SQL dump generation. This setup ensures modularity, security, and compatibility with the project structure, enabling seamless execution of database operations.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..')) 

from dotenv import load_dotenv
import oracledb
from scripts.db_insert import insert_reviews_into_oracle
from scripts.sql_dumper import generate_sql_dump

### Load Database Credentials

Credentials are loaded from a `.env` file to establish a connection to the Oracle XE database. This approach prevents exposing sensitive information in the notebook or version-controlled files.

In [2]:
load_dotenv()
user = os.getenv("ORACLE_USER")
password = os.getenv("ORACLE_PASSWORD")
dsn = os.getenv("ORACLE_DSN")

### Insert Reviews into Oracle Database

The `insert_reviews_into_oracle` function from `db_insert.py` loads cleaned review data from CSV files in the `../data` directory, inserts it into the `banks` and `reviews` tables, and commits the transaction. The database schema was defined as:

- **Banks Table**:
  - `bank_id`: Auto-incrementing primary key.
  - `bank_name`: Unique name of the bank (e.g., "CBE").
- **Reviews Table**:
  - `review_id`: Auto-incrementing primary key.
  - `bank_id`: Foreign key referencing `banks(bank_id)`.
  - `review_text`: Review content (CLOB).
  - `rating`: User rating (1–5).
  - `review_date`: Date of the review.
  - `source`: Review source (e.g., "Google Play").
  - `sentiment_label`: Sentiment classification (positive, negative, neutral).
  - `sentiment_score`: Sentiment score from VADER.
  - `keywords`: Extracted keywords (CLOB).
  - `themes`: Identified themes (CLOB).

In [3]:
insert_reviews_into_oracle(user, password, dsn)

✅ All reviews and bank data inserted into Oracle successfully.


### Validate Data Insertion

To verify the success of the data insertion we query the row counts for the `banks` and `reviews` tables.
  - Establishes a new database connection to query the number of rows in each table.
  - Uses `SELECT COUNT(*)` to count rows in `banks` and `reviews`.
  - Closes the cursor and connection to free resources.
  - Prints the row counts for validation.
  
**Why**: This step ensures the database is correctly populated before generating the SQL dump.

In [4]:
conn = oracledb.connect(user=user, password=password, dsn=dsn, mode=oracledb.SYSDBA)
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM banks")
bank_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM reviews")
review_count = cursor.fetchone()[0]

cursor.close()
conn.close()

print(f"\nBanks table: {bank_count} rows")
print(f"Reviews table: {review_count} rows")


Banks table: 3 rows
Reviews table: 1432 rows


**Results**:
  - **Banks table (3 rows)**: Indicates three unique banks were inserted, consistent with processing multiple bank-specific CSV files.
  - **Reviews table (1432 rows)**: Exceeds >1,000 entries, confirming the pipeline successfully stored a substantial dataset.

---

### Generate SQL Dump for GitHub

To ensure reproducibility the `generate_sql_dump` function from `generate_sql_dump.py` creates a `.sql` file containing the table schema (DDL) and data (INSERT statements).
- Connects to the Oracle database to extract the schema for `banks` and `reviews` using `DBMS_METADATA.GET_DDL`.
  - Queries all rows from both tables to generate `INSERT` statements.
  - Handles `CLOB` columns (`review_text`, `keywords`, `themes`) by converting `LOB` objects to strings.
  - Writes the DDL and INSERT statements to `../SQL_dump/database_dump1.sql`, creating the `SQL_dump` directory if needed.

**Why**: The SQL dump allows the database state to be version-controlled on GitHub, enabling team collaboration and reproducibility. It captures the exact schema and data for future reference or deployment.

**Details**:
  - The schema includes table definitions, primary keys, foreign keys, and column types (e.g., `CLOB`, `DATE`).
  - Data is exported with proper escaping (e.g., single quotes in text fields) and formatting (e.g., `TO_DATE` for dates).
  - The output file is saved in a dedicated `SQL_dump` directory for organization.

In [ ]:
generate_sql_dump(user, password, dsn)

✅ SQL dump written to ../SQL dump/database_dump.sql


---

## Conclusion

This notebook successfully completes the task of storing cleaned Google Play review data in an Oracle XE database. Key achievements include:
- Establishing a robust connection to Oracle XE using `oracledb`.
- Inserting 3 banks and 1,432 reviews.
- Validating the data insertion with row count queries.
- Generating an SQL dump (`../SQL_dump/database_dump.sql`) for version control on GitHub.

The pipeline demonstrates enterprise-grade data engineering practices, including secure credential management, modular scripting, and data validation.